In [1]:
import polars as pl
import time

import sys
sys.path.insert(0, './star-charts')
import sky_area
from star_data import StarData, StarDataList
from input_file import InputFile
from diagram import Diagram
from coord_calc import CoordCalc

In [2]:
dftycho = pl.read_parquet('./support/tyc2-3.parquet')
dftycho.describe()

describe,RAmdeg,DEmdeg,Vmag
str,f64,f64,f64
"""count""",2.430468e6,2.430468e6,2.430468e6
"""null_count""",0.0,0.0,0.0
"""mean""",188.530502,-3.479685,11.071772
"""std""",100.340485,41.43108,1.108483
"""min""",0.000339,-89.889664,1.89558
"""25%""",105.471535,-39.00515,10.54628
"""50%""",186.052124,-4.889748,11.292219
"""75%""",280.987732,31.3225,11.82646
"""max""",359.999878,89.832336,15.37354


In [3]:
def stars_for_point_and_radius(df, point, radius, max_mag):
    """ point is in the corner, not the center """
    ra, dec = point
    minra = ra
    maxra = ra + radius
    mindec = dec
    maxdec = dec + radius
    return df.filter((pl.col("RAmdeg") < maxra) & (pl.col("RAmdeg") > minra) & (pl.col("DEmdeg") < maxdec) & (pl.col("DEmdeg") > mindec) & (pl.col("Vmag") <= max_mag))

def get_grid_points(min_dec=-90, max_dec=90):
    RA_values = [ra for ra in range(0, 361, 4)]  # Increment by 4 for a 2-degree radius
    Dec_values = [dec for dec in range(min_dec, max_dec+1, 4)]  # Increment by 4 for a 2-degree radius
    grid_points = [(ra, dec) for ra in RA_values for dec in Dec_values]
    return grid_points

def get_grid_point_by_idx(idx):
    gp = get_grid_points()
    return gp[idx]

def get_region(dftycho, idx, radius, min_dec, max_dec, max_mag):
    gp = get_grid_points(min_dec, max_dec)
    point = gp[idx]
    return stars_for_point_and_radius(dftycho, point, radius, max_mag)


def draw_region(region, regionstars, focusstars, label):
    area  = sky_area.SkyArea(regionstars['RAmdeg'].min(),regionstars['RAmdeg'].max(), regionstars['DEmdeg'].min(), regionstars['DEmdeg'].max(), regionstars['Vmag'].min())
    print(str(area))
    star_data_list = [StarData(x[0], x[1], x[2]) for x in regionstars.rows()]
    for star in focusstars:
        star_data_list.append(StarData(star[0], star[1], star[2], label))
    star_data_list = StarDataList(star_data_list)
    
    cc = CoordCalc(star_data_list, area, 400)    
    cc.process()
    d = Diagram(f'Region {region}', area, star_data_list)
    list(map(d.add_curve, cc.calc_curves()))     
    d.render_svg(f'region_{region}.svg')

In [4]:
result_filename = 'result_squareness.parquet'
df = pl.read_parquet(result_filename)
print("length of parquet results:", len(df))
head = df.sort("score1").head(10)
print(head)
chosen = head[0]
chosen

length of parquet results: 220
shape: (10, 5)
┌──────────┬────────┬──────────┬──────────┬───────────────────────────────────┐
│ score    ┆ region ┆ score1   ┆ score2   ┆ stars                             │
│ ---      ┆ ---    ┆ ---      ┆ ---      ┆ ---                               │
│ f64      ┆ i64    ┆ f64      ┆ f64      ┆ list[list[f64]]                   │
╞══════════╪════════╪══════════╪══════════╪═══════════════════════════════════╡
│ 1.008304 ┆ 65     ┆ 0.008773 ┆ 0.999531 ┆ [[8.364757, -57.518147, 11.49566… │
│ 1.00867  ┆ 65     ┆ 0.00882  ┆ 0.999849 ┆ [[8.814655, -57.732128, 11.71213… │
│ 1.00863  ┆ 96     ┆ 0.008909 ┆ 0.999721 ┆ [[12.873372, -62.116669, 12.0100… │
│ 1.008673 ┆ 96     ┆ 0.008952 ┆ 0.999721 ┆ [[12.873372, -62.116669, 12.0100… │
│ …        ┆ …      ┆ …        ┆ …        ┆ …                                 │
│ 1.008676 ┆ 64     ┆ 0.010367 ┆ 0.998309 ┆ [[9.53124, -61.70174, 11.82692],… │
│ 1.010283 ┆ 1      ┆ 0.010593 ┆ 0.99969  ┆ [[1.627652, -57.284172, 11.393

score,region,score1,score2,stars
f64,i64,f64,f64,list[list[f64]]
1.008304,65,0.008773,0.999531,"[[8.364757, -57.518147, 11.49566], [9.546021, -58.045547, 10.99091], [9.496355, -58.370415, 11.99044]]"


In [5]:
#region = df.sort("score")[0]['region'][0]
region = int(chosen['region'][0])
print(region)
region_stars = get_region(dftycho, region, 2, -63,63, 15)
print(region_stars.describe())

65
shape: (9, 4)
┌────────────┬──────────┬────────────┬───────────┐
│ describe   ┆ RAmdeg   ┆ DEmdeg     ┆ Vmag      │
│ ---        ┆ ---      ┆ ---        ┆ ---       │
│ str        ┆ f64      ┆ f64        ┆ f64       │
╞════════════╪══════════╪════════════╪═══════════╡
│ count      ┆ 75.0     ┆ 75.0       ┆ 75.0      │
│ null_count ┆ 0.0      ┆ 0.0        ┆ 0.0       │
│ mean       ┆ 9.021595 ┆ -57.903736 ┆ 10.982916 │
│ std        ┆ 0.494077 ┆ 0.561898   ┆ 1.22158   │
│ min        ┆ 8.057153 ┆ -58.990746 ┆ 6.81971   │
│ 25%        ┆ 8.672735 ┆ -58.33675  ┆ 10.27339  │
│ 50%        ┆ 8.960945 ┆ -57.825565 ┆ 11.30423  │
│ 75%        ┆ 9.41347  ┆ -57.481609 ┆ 11.9509   │
│ max        ┆ 9.944145 ┆ -57.046745 ┆ 12.71198  │
└────────────┴──────────┴────────────┴───────────┘


In [6]:
focusstars = chosen['stars'].to_list()[0]
print(focusstars)
draw_region(region, region_stars, focusstars, "a")

time.sleep(5)
!convert -size 800x600 region_{region}.svg region_{region}.png
!open region_{region}.png

[[8.36475658416748, -57.51814651489258, 11.495659828186035], [9.546021461486816, -58.04554748535156, 10.990909576416016], [9.496355056762695, -58.37041473388672, 11.990440368652344]]
SkyArea: ra_min=8.057153 ra_max=9.944145 dec_min=-58.990746 dec_max=-57.046745 mag_min=6.819710
